In [12]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [1]:
import re
with open('models/research/object_detection/packages/tf2/setup.py') as f:
  s = f.read()

  with open('models/research/setup.py', 'w') as f:
  #fine tune check-point
    s = re.sub('tf-models-official>=2.5.1',
             'tf-models-official==2.8.0', s)
    f.write(s)

In [2]:
!export LD_LIBRARY_PATH=/user/local/cuda-11.2/lib64:$LD_LIBRARY_PATH

In [3]:
!python models/research/object_detection/builders/model_builder_tf2_test.py

2024-03-08 15:20:51.659383: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-08 15:20:51.659635: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-08 15:20:51.677333: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-08 15:20:51.677579: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-08 15:20:51.677783: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

In [4]:
# !mkdir images

In [5]:
# !unzip -q images.zip -d images/all

In [6]:
# !mkdir images/train

In [7]:
# !mkdir images/validation
# !mkdir images/test

In [8]:
#tf obdetector dataset caller

from pathlib import Path
import random
import os
import sys


image_path = 'images/all'
train_path = 'images/train'
val_path = 'images/validation'
test_path = 'images/test'




jpeg_file_list = [path for path in Path(image_path).rglob('*.jpeg')]
jpg_file_list =  [path for path in Path(image_path).rglob('*.jpg')]
png_file_path =  [path for path in Path(image_path).rglob('*.png')]
bmp_file_list =  [path for path in Path(image_path).rglob('*.bmp')]

if sys.platform == 'linux':
  JPEG_file_list = [path for path in Path(image_path).rglob('*.JPEG')]
  JPG_file_list = [path for path in Path(image_path).rglob('*.JPG')]
  file_list = jpg_file_list = jpg_file_list + JPG_file_list + png_file_path + bmp_file_list + JPEG_file_list + jpeg_file_list

else:
  file_list = jpg_file_list + png_file_list + bmp_file_list + jpeg_file_list


file_num = len(file_list)
print ('Total_image: %d' % file_num)





train_per = 0.80
val_per = 0.10
test_per = 0.1
train_num = int(file_num*train_per)
val_num= int(file_num*val_per)
# test_num = int(file_num*test_per)
test_num = file_num - train_num - val_num

# test_num = file_num - train_num - val_num

print ('Image for train setted: %d' % train_num)
print ('Image for validation: %d' %val_num)
print ('Image for testing: %d' %test_num)


#selection of train in random
#my choice to move
for i in range(train_num):
  move_me = random.choice(file_list)
  fn = move_me.name #extracting the name from file list
  base_fn = move_me.stem #extracting without extension
  parent_path = move_me.parent #get the parent path
  xml_fn = base_fn + '.xml'
  os.rename(move_me, train_path+'/'+fn)
  os.rename(os.path.join(parent_path,xml_fn),os.path.join(train_path,xml_fn))
  file_list.remove(move_me)


#select for val test

for i in range(val_num):
  move_me = random.choice(file_list)
  fn = move_me.name
  base_fn =  move_me.stem
  parent_path = move_me.parent
  xml_fn = base_fn + '.xml'
  os.rename(move_me, val_path+'/'+fn)
  os.rename(os.path.join(parent_path,xml_fn),os.path.join(val_path,xml_fn))
  file_list.remove(move_me)

#remeining file to test

for i in range(test_num):
  move_me = random.choice(file_list)
  fn = move_me.name
  base_fn = move_me.stem
  parent_path = move_me.parent
  xml_fn = base_fn + '.xml'
  os.rename(move_me, test_path+'/'+fn)
  os.rename(os.path.join(parent_path,xml_fn),os.path.join(test_path,xml_fn))
  file_list.remove(move_me)

Total_image: 0
Image for train setted: 0
Image for validation: 0
Image for testing: 0


In [9]:
# %%bash
# cat <<EOF >> labelmap.txt
# product

In [10]:
# #create_xml.py

# import os
# import glob
# import pandas as pd
# import xml.etree.ElementTree as ET

# def xml_to_csv(path):
#   xml_list = []
#   for xml_file in glob.glob(path + '/*.xml'):
#     tree = ET.parse(xml_file)
#     root = tree.getroot()
#     for member in root.findall('object'):
#       value = (root.find('filename').text,
#                int(root.find('size')[0].text),
#                int(root.find('size')[1].text),
#                member[0].text,
#                int(member[4][0].text),
#                int(member[4][1].text),
#                int(member[4][2].text),
#                int(member[4][3].text)
#                )
#       xml_list.append(value)

#     column_name = ['filename', 'width','height','class','xmin','ymin','xmax','ymax']
#     xml_df = pd.DataFrame(xml_list, columns=column_name)
#     return xml_df

# def main():
#   for folder in ['train','validation']:
#     image_path = os.path.join(os.getcwd(), ('kumaImageV1/' + folder))
#     xml_df = xml_to_csv(image_path)
#     xml_df.to_csv(('kumaImageV1/' + folder + '_labels.csv'), index=None)
#     print('Successfully converted xml to csv file')

# main()


In [11]:
# #creating_tfrecord.py

# #from tfmodels/to create model data

# # create traintest data
# #command: python generate_tfrecord.py --csv_input=images/train_labels.csv --image_dir=images/train --output_path=train.record
# #command: python generate_tfrecord.py --csv_input=image/test_labels.csv --image_dir=images/test --output_path=test.record
# #importing division means taking

# from __future__ import division
# from __future__ import print_function
# from __future__ import absolute_import

# import os
# import io
# import pandas as pd

# from tensorflow.python.framework.versions import VERSION
# if VERSION >="2.0.0a0":
#   import tensorflow.compat.v1 as tf
# else:
#   import tensorflow as tf

# from PIL import Image
# from object_detection.utils import dataset_util
# from collections import namedtuple, OrderedDict


# flags = tf.app.flags
# flags.DEFINE_string('csv_input', '', 'path to the csv input')
# flags.DEFINE_string('labelmap', '', 'path to the labelmap file')
# flags.DEFINE_string('image_dir', '', 'path to the image directoiry')
# flags.DEFINE_string('output_path', '', 'path to the output TFRecord')
# FLAGS = flags.FLAGS


# def split(df,group):
#   data = namedtuple('data', ['filename','object'])
#   gb = df.groupby(group)
#   return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


# def create_tf_example(group,path):
#   with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
#     encoded_jpg = fid.read()
#   encoded_jpg_io = io.BytesIO(encoded_jpg)
#   image = Image.open(encoded_jpg_io)
#   width, height = image.size


#   filename = group.filename.encode('utf8')
#   image_format = b'jpg'
#   xmins = []
#   xmaxs = []
#   ymins = []
#   ymaxs = []
#   classes_text = []
#   classes = []

#   labels = []
#   with open(FLAGS.labelmap, 'r') as f:
#     labels = [line.strip() for line in f.readlines()]

#   for index, row in group.object_detection.iterrows():
#     xmins.append(row['xmin'] / width)
#     xmaxs.append(row['xmax'] / width)
#     ymins.append(row['ymins'] / height)
#     ymaxs.append(row['ymax'] / height)
#     classes_text.append(row['class'].encode('utf8'))
#     classes_text.append(int(labels.index(row['class'])+1))

#   tf_example = tf.train.Example(features=tf.train.Features(feature={
#       'image/height': dataset_util.int64_feature(height),
#       'image/width' : dataset_util.int64_feature(width),
#       'image/filename' : dataset_util.bytes_feature(filename),
#       'image/source_id' : dataset_util.bytes_feature(filename),
#       'image/encoded' : dataset_util.bytes_feature(encoded_jpg),
#       'image/format': dataset_util.bytes_feature(image_format),
#       'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
#       'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
#       'image/object/bbox/ymin' : dataset_util.float_list_feature(ymins),
#       'image/object/bbox/ymax' : dataset_util.float_list_feature(ymaxs),
#       'image/object/class/text': dataset_util.float_list_feature(classes_text),
#       'image/object/class/label': dataset_util.float_list_feature(classes)

#   }))
#   return tf_example


# def main(_):
#   #loadprep
#   writer = tf.python_io.TFRecordWriter(FLAGS.output_path)
#   path = os.path.join(os.getcwd(), FLAGS.image_dir)
#   examples = pd.read_csv(FLAGS.csv_input)
#   #tfRF
#   grouped = split(examples, 'filename')
#   for group in grouped:
#     tf_example = create_tf_example(group, path)
#     writer.write(tf_example.SerializeToString())

#   writer.close()
#   output_path = os.path.join(os.getcwd(), FLAGS.output_path)
#   print('Successfully created the tf record')


#   #labelmap
#   path_to_labeltext = os.path.join(os.getcwd(), FLAGs.labelmap)
#   with open(path_to_labeltxt, 'r') as f:
#     labels = [line.strip() for line in f.readlines()]

#   path_to_labelpbtxt = os.path.join(os.getcwd(), 'labelmap.pbtxt')
#   with open(path_to_labelpbtxt,'w') as f:
#     for i, label in enumerate(labels):
#       f.write('item {\n' +
#               ' id: %d\n' % (i+1) +
#               ' name: \'%s\'\n' % label +
#               '}\n' +
#               '\n')


# if __name__ == '__main__':
#   tf.app.run()

In [1]:
# !python3 create_csv.py
# !python3 create_tfrecord.py --csv_input=images/train_labels.csv --labelmap=labelmap.txt --image_dir=images/train --output_path=train.tfrecord
# !python3 create_tfrecord.py --csv_input=images/validation_labels.csv --labelmap=labelmap.txt --image_dir=images/validation --output_path=val.tfrecord

In [13]:
train_record_fname = './train.tfrecord'
val_record_fname = './val.tfrecord'
label_map_pbtxt_fname = './labelmap.pbtxt'

In [14]:
#tranning


# Change the chosen_model variable to deploy different models available in the TF2 object detection zoo
chosen_model = 'ssd-mobilenet-v2-fpnlite-320'

MODELS_CONFIG = {
    'ssd-mobilenet-v2': {
        'model_name': 'ssd_mobilenet_v2_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz',
    },
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
    },
    'ssd-mobilenet-v2-fpnlite-320': {
        'model_name': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz',
    },

    #'centernet-mobilenet-v2': {
    #    'model_name': 'centernet_mobilenetv2fpn_512x512_coco17_od',
    #    'base_pipeline_file': 'pipeline.config',
    #    'pretrained_checkpoint': 'centernet_mobilenetv2fpn_512x512_coco17_od.tar.gz',
    #}
}

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']

In [15]:
# %mkdir models/mymodel

In [16]:
# !ls models/mymodel

In [17]:
# %cd models/mymodel/

# # pre-trained model weights
# import tarfile
# download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint
# !wget {download_tar}
# tar = tarfile.open(pretrained_checkpoint)
# tar.extractall()
# tar.close()

# # training configuration file for model
# download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
# !wget {download_config}

In [18]:
num_steps = 4000

if chosen_model == 'efficientdet-d0':
  batch_size = 4
else:
  batch_size = 4

In [2]:
# !pwd

In [20]:
pipeline_fname = '/mymodel/' + base_pipeline_file
fine_tune_checkpoint = '/mymodel' + model_name + '/checkpoint/ckpt-0'
# label_map_pbtxt_fname = 'labelmap.pbtxt'
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())
num_classes = get_num_classes(label_map_pbtxt_fname)
print('Total classes:', num_classes)

Total classes: 1


In [3]:
# dummy_path = './models/mymodel/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config'

In [23]:
# Create custom configuration file by writing the dataset, model checkpoint, and training parameters into the base pipeline file
import re

# %cd /content/models/mymodel
print('writing custom configuration file')

pipeline_fname = "./models/mymodel/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config"

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:

    # Set fine_tune_checkpoint path
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)

    # Set tfrecord files for train and test datasets
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(val_record_fname), s)

    # Set label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set batch_size
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)

    # Set number of classes num_classes
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)

    # Change fine-tune checkpoint type from "classification" to "detection"
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)

    # If using ssd-mobilenet-v2, reduce learning rate (because it's too high in the default config file)
    if chosen_model == 'ssd-mobilenet-v2':
      s = re.sub('learning_rate_base: .08',
                 'learning_rate_base: .008', s)

      s = re.sub('warmup_learning_rate: 0.13333',
                 'warmup_learning_rate: .026666', s)

    # If using efficientdet-d0, use fixed_shape_resizer instead of keep_aspect_ratio_resizer (because it isn't supported by TFLite)
    if chosen_model == 'efficientdet-d0':
      s = re.sub('keep_aspect_ratio_resizer', 'fixed_shape_resizer', s)
      s = re.sub('pad_to_max_dimension: true', '', s)
      s = re.sub('min_dimension', 'height', s)
      s = re.sub('max_dimension', 'width', s)

    f.write(s)

writing custom configuration file


In [24]:
# custom configuration file's contents
!cat /home/protiva/models/mymodel/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config

# SSD with Mobilenet v2 FPN-lite (go/fpn-lite) feature extractor, shared box
# predictor and focal loss (a mobile version of Retinanet).
# Retinanet: see Lin et al, https://arxiv.org/abs/1708.02002
# Trained on COCO, initialized from Imagenet classification checkpoint
# Train on TPU-8
#
# Achieves 22.2 mAP on COCO17 Val

model {
  ssd {
    inplace_batchnorm_update: true
    freeze_batchnorm: false
    num_classes: 90
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    encode_background_as_zeros: true
    anchor_generator {
      multiscale_anchor_generator 

In [32]:
pipeline_file = './models/mymodel/pipeline_file.config'
model_dir = './training/'

In [26]:
%load_ext tensorboard
%tensorboard --logdir 'training/train'

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2024-03-08 15:21:18.549913: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-08 15:21:18.550168: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-08 15:21:18.567314: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-08 15:21:18.567585: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-08 15:21:18.567780: I tensorflow/stream

In [27]:
# /home/protiva/models

In [29]:
!sudo python /home/protiva/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1

[sudo] protiva のパスワード: 


In [34]:
import getpass
import subprocess

# Get the sudo password securely
sudo_password = getpass.getpass(prompt='Enter your sudo password: ')

# Define the command to run with sudo
command = f"sudo python /home/protiva/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1"

# Run the command with sudo using subprocess
subprocess.run(command, shell=True, input=sudo_password.encode(), check=True)


Enter your sudo password:  ········


sudo: a terminal is required to read the password; either use the -S option to read from standard input or configure an askpass helper


CalledProcessError: Command 'sudo python /home/protiva/models/research/object_detection/model_main_tf2.py     --pipeline_config_path=/path/to/pipeline.config     --model_dir=/path/to/model_directory     --alsologtostderr     --num_train_steps=1000     --sample_1_of_n_eval_examples=1' returned non-zero exit status 1.

In [1]:
!python /home/protiva/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path='./models/mymodel/pipeline_file.config' \
    --model_dir='./protivatraining/' \
    --alsologtostderr \
    --num_train_steps=40000 \
    --sample_1_of_n_eval_examples=1 \

Traceback (most recent call last):
  File "/home/protiva/models/research/object_detection/model_main_tf2.py", line 30, in <module>
    import tensorflow.compat.v2 as tf
  File "/home/protiva/ahmed/lib/python3.8/site-packages/tensorflow/__init__.py", line 37, in <module>
    from tensorflow.python.tools import module_util as _module_util
  File "/home/protiva/ahmed/lib/python3.8/site-packages/tensorflow/python/__init__.py", line 45, in <module>
    from tensorflow.python.feature_column import feature_column_lib as feature_column
  File "/home/protiva/ahmed/lib/python3.8/site-packages/tensorflow/python/feature_column/feature_column_lib.py", line 18, in <module>
    from tensorflow.python.feature_column.feature_column import *
  File "/home/protiva/ahmed/lib/python3.8/site-packages/tensorflow/python/feature_column/feature_column.py", line 143, in <module>
    from tensorflow.python.layers import base
  File "/home/protiva/ahmed/lib/python3.8/site-packages/tensorflow/python/layers/base.py"